# Recognizing Brand & Models

In [1]:
import pandas as pd

In [2]:
models = pd.read_csv('models.csv', header = None, names = ['Brand', 'Model'])

**Delete Duplicates**

In [3]:
models['Brand'] = models['Brand'].str.lower()
models['Model'] = models['Model'].str.lower()
models['Combined'] = (models['Brand'] + ' ' + models['Model'])
models['Combined'] = models['Combined'].str.replace('[\s\-,._]', '', regex = True)

unique_model = models.drop_duplicates(subset = 'Combined')
unique_model = unique_model.drop(columns = ['Combined'])

**Check Brand list**

In [4]:
brand = unique_model['Brand'].unique().tolist()
print(brand)

['acura', 'audi', 'bmw', 'buick', 'cadillac', 'car', 'chevrolet', 'chrysler', 'dodge', 'ford', 'honda', 'hyndai kia', 'hyundai', 'hyundai,', 'infiniti', 'kia', 'lincoln', 'mazda', 'mercedes', 'mercury', 'mitsubishi', 'nissan', 'nissan.', 'pontiac', 'problem', 'saturn', 'seat', 'sedan', 'subaru', 'suzuki', 'toyata', 'toyota', 'volkswagen', 'volkwagen', 'volvo']


**Notice weird brands:**

- _hyundai kia_, _car_, _problem_, _seat_ should be deleted
- _hyndai kia_ should be _hyundai_
- _hyundai,_ and _nissan._ should not have ',' and '.'.
- _toyata_ should be _toyota_
- _volkwagen_ should be _volkswagen_

In [5]:
new_model = unique_model[~unique_model['Brand'].isin(['hyundai kia', 'car', 'problem', 'seat'])]

new_model.loc[new_model['Brand'].str.contains('hyundai') & ~ new_model['Brand'].eq('hyundai'), 'Brand'] = 'hyundai'
new_model.loc[new_model['Brand'].str.contains('kia') & ~ new_model['Brand'].eq('kia'), 'Brand'] = 'kia'
new_model.loc[new_model['Brand'].str.contains('nissan') & ~ new_model['Brand'].eq('nissan'), 'Brand'] = 'nissan'
new_model.loc[new_model['Brand'].eq('toyata'), 'Brand'] = 'toyota'
new_model.loc[new_model['Brand'].eq('volkwagen'), 'Brand'] = 'volkswagen'

**Delete Dup Again**

In [6]:
new_model['Combined'] = (new_model['Brand'] + ' ' + new_model['Model'])
new_model['Combined'] = new_model['Combined'].str.replace('[\s\-,._]', '', regex = True)

new_model = new_model.drop_duplicates(subset = 'Combined')
new_model = new_model.drop(columns = ['Combined'])

/var/folders/z4/djkh1_vd3bb6xf3hy80b_l9m0000gn/T/ipykernel_64036/354213253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_model['Combined'] = (new_model['Brand'] + ' ' + new_model['Model'])
/var/folders/z4/djkh1_vd3bb6xf3hy80b_l9m0000gn/T/ipykernel_64036/354213253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_model['Combined'] = new_model['Combined'].str.replace('[\s\-,._]', '', regex = True)


**Check brand**

In [7]:
brand_list = new_model['Brand'].unique().tolist()
print(brand_list)

['acura', 'audi', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'dodge', 'ford', 'honda', 'kia', 'hyundai', 'infiniti', 'lincoln', 'mazda', 'mercedes', 'mercury', 'mitsubishi', 'nissan', 'pontiac', 'saturn', 'sedan', 'subaru', 'suzuki', 'toyota', 'volkswagen', 'volvo']


In [8]:
model_list = new_model['Model'].unique().tolist()
print(model_list)

['integra', 'legend', 'vigor', 'rlx', 'ilx', 'mdx', 'rdx', 'tsx', 'rsx', 'quattro', 'a4', 'a6', 'a5', 'a7', 'q3', 'q5', 'q7', 'a8', 'a3', '3-series', '5-series', '6-series', '4-series', 'xdrive', 'alpina', '330ci', '328i', '330c', '335i', '335d', '528i', '535i', '128i', '640i', '3er', '5er', 'm5', 'm3', 'm6', 'bmws', 'bmwesque', 'park avenue', 'la crosse', 'century', 'enclave', 'encore', 'regal', 'buicks', 'escalade', 'srx', 'xts', 'grand blazer', 'trailblazer', 'silverado', 'colorado', 'corvette', 'suburban', 'cavalier', 'caprice', 'captiva', 'equinox', 'express', 'lacetti', 'montana', 'uplander', 'orlando', 'tornado', 'tracker', 'traverse', 'camaro', 'cobalt', 'impala', 'lumina', 'malibu', 'prisma', 'tavera', 'agile', 'celta', 'cruze', 'lanos', 'omega', 'optra', 'sonic', 'spark', 'tahoe', 'aveo', 'lova', 'n200', 'niva', 'onix', 's-10', 'sail', 'volt', 'chevy', 'malibus', 'town and country', 'chrysler 300', 'chrysler 200', 'pacifica', 'cruiser', 'sebring', '300m', 'lx', 'chryslers', '

**Check model**

In [9]:
print(len(new_model))
print(new_model['Model'].nunique())
duplicated_model = new_model[new_model.duplicated(subset = 'Model', keep = False)]
print("Rows with duplicate values in 'Model' column:")
print(duplicated_model)

481
478
Rows with duplicate values in 'Model' column:
       Brand    Model
1      acura   legend
44     buick  century
171    honda   legend
224  hyundai   matrix
420   toyota  century
444   toyota   matrix


**Modify model dup & create dictionary for models**

In [10]:
modify_rows = [1,44,171, 224, 420, 444]
new_model.loc[modify_rows, 'Model'] = new_model.loc[modify_rows, 'Brand'] + ' ' + new_model.loc[modify_rows, 'Model']
print(new_model['Model'].nunique())
model_dict = new_model.set_index('Model')['Brand'].to_dict()

481


# Load All Comments

In [11]:
comments = pd.read_csv('AllComments.csv', on_bad_lines='skip')

In [12]:
comments.head(3) # see what the comments look like

,User Name,Comment Date,Full Comment
0,bdreggors,June 2000,One of my current favorites is the Lexus IS c...
1,markz2k,June 2000,What's the deal with the Dog family in the Od...
2,roadroach,June 2000,The whole point of commercials is to get your...


In [13]:
comments.shape[0] # number of rows int the file

8050

**Note: some rows are skipped due to the formatting, but there are more than 5K lines of comments**

## Tokenization

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize

comments['Full Comment'] = comments['Full Comment'].apply(lambda x: str(x).lower())
comments['Sentences'] = comments['Full Comment'].apply(lambda x: sent_tokenize(str(x)))
comments['Words'] = comments['Full Comment'].apply(lambda x: word_tokenize(str(x)))

comments.head(3)

,User Name,Comment Date,Full Comment,Sentences,Words
0,bdreggors,June 2000,one of my current favorites is the lexus is c...,[ one of my current favorites is the lexus is ...,"[one, of, my, current, favorites, is, the, lex..."
1,markz2k,June 2000,what's the deal with the dog family in the od...,[ what's the deal with the dog family in the o...,"[what, 's, the, deal, with, the, dog, family, ..."
2,roadroach,June 2000,the whole point of commercials is to get your...,[ the whole point of commercials is to get you...,"[the, whole, point, of, commercials, is, to, g..."


# Recognizing Brand & Models

In [15]:
def extract_brands_models(words):
    brand_part = []
    model_part = []

    for word in words:
        if word in brand_list and word not in brand_part:
            brand_part.append(word)
        elif word in model_list and word not in model_part:
            model_part.append(word)

    return [brand_part, model_part]

comments['Contain Brand_Model'] = comments['Words'].apply(extract_brands_models)
comments.head(3)

,User Name,Comment Date,Full Comment,Sentences,Words,Contain Brand_Model
0,bdreggors,June 2000,one of my current favorites is the lexus is c...,[ one of my current favorites is the lexus is ...,"[one, of, my, current, favorites, is, the, lex...","[[toyota], []]"
1,markz2k,June 2000,what's the deal with the dog family in the od...,[ what's the deal with the dog family in the o...,"[what, 's, the, deal, with, the, dog, family, ...","[[], [accord]]"
2,roadroach,June 2000,the whole point of commercials is to get your...,[ the whole point of commercials is to get you...,"[the, whole, point, of, commercials, is, to, g...","[[toyota], [tundra]]"


## Recognize Unique Brand

In [22]:
def extract_unique_brands(brand_model_list):
    unique_brands = set()
    for brand in brand_model_list[0]:
        unique_brands.add(brand)
    for model in brand_model_list[1]:
        brand_from_dict = model_dict.get(model)
        if brand_from_dict not in unique_brands and brand_from_dict is not None:
            unique_brands.add(brand_from_dict)

    return list(unique_brands)

comments['Unique Brand'] = comments['Contain Brand_Model'].apply(extract_unique_brands)
comments.head(20)


,User Name,Comment Date,Full Comment,Sentences,Words,Contain Brand_Model,Unique Brand
0,bdreggors,June 2000,one of my current favorites is the lexus is c...,[ one of my current favorites is the lexus is ...,"[one, of, my, current, favorites, is, the, lex...","[[toyota], []]",[toyota]
1,markz2k,June 2000,what's the deal with the dog family in the od...,[ what's the deal with the dog family in the o...,"[what, 's, the, deal, with, the, dog, family, ...","[[], [accord]]",[honda]
2,roadroach,June 2000,the whole point of commercials is to get your...,[ the whole point of commercials is to get you...,"[the, whole, point, of, commercials, is, to, g...","[[toyota], [tundra]]",[toyota]
3,markz2k,June 2000,"yeah, i agree the point is to get your attent...","[ yeah, i agree the point is to get your atten...","[yeah, ,, i, agree, the, point, is, to, get, y...","[[honda], [tundra, accord]]","[honda, toyota]"
4,bdreggors,June 2000,i've noticed two fallicies in hodna commercia...,[ i've noticed two fallicies in hodna commerci...,"[i, 've, noticed, two, fallicies, in, hodna, c...","[[honda], [accord, accords, civic]]",[honda]
5,floridian,June 2000,the camry commercial that says over and over ...,[ the camry commercial that says over and over...,"[the, camry, commercial, that, says, over, and...","[[], [camry]]",[toyota]
6,fezo,June 2000,absolutely! next chevy will advertise sexy ma...,"[ absolutely!, next chevy will advertise sexy ...","[absolutely, !, next, chevy, will, advertise, ...","[[], [chevy, malibus]]",[chevrolet]
7,bdreggors,June 2000,that's an oxymoron,[ that's an oxymoron],"[that, 's, an, oxymoron]","[[], []]",[]
8,microrepair,June 2000,is totally irritated by those toyotathon comm...,[ is totally irritated by those toyotathon com...,"[is, totally, irritated, by, those, toyotathon...","[[toyota], [tundra]]",[toyota]
9,andrew21,June 2000,"...the new ""zoom zoom zoom"" campaign by toyot...","[ ...the new ""zoom zoom zoom"" campaign by toyo...","[..., the, new, ``, zoom, zoom, zoom, '', camp...","[[toyota, bmw], []]","[bmw, toyota]"


## Frequency of Brand Mentions

In [23]:
brand_in_comments = [brand for brands_list in comments['Unique Brand'] for brand in brands_list if brand]

unique_brand_count = {}
for i in brand_in_comments:
    if i not in unique_brand_count:
        unique_brand_count[i] = 0
    else:
        pass
    unique_brand_count[i] += 1

brand_count_df = pd.DataFrame(list(unique_brand_count.items()), columns=['Brand', 'Count']).sort_values(by='Count', ascending=False)
brand_count_df.head(10)

,Brand,Count
0,toyota,897
1,honda,558
4,ford,506
2,chevrolet,480
5,volkswagen,408
13,hyundai,350
11,subaru,296
6,nissan,267
3,bmw,258
10,mercedes,246


# Task A

## Identify top 10 brands by frequency

In [24]:
top10brand = brand_count_df['Brand'].head(10).tolist()
top10brand

['toyota',
 'honda',
 'ford',
 'chevrolet',
 'volkswagen',
 'hyundai',
 'subaru',
 'nissan',
 'bmw',
 'mercedes']

In [25]:
filter_comments = comments[comments['Unique Brand'].apply(lambda x: any(brand in top10brand for brand in x))]
filter_comments.head(15)

,User Name,Comment Date,Full Comment,Sentences,Words,Contain Brand_Model,Unique Brand
0,bdreggors,June 2000,one of my current favorites is the lexus is c...,[ one of my current favorites is the lexus is ...,"[one, of, my, current, favorites, is, the, lex...","[[toyota], []]",[toyota]
1,markz2k,June 2000,what's the deal with the dog family in the od...,[ what's the deal with the dog family in the o...,"[what, 's, the, deal, with, the, dog, family, ...","[[], [accord]]",[honda]
2,roadroach,June 2000,the whole point of commercials is to get your...,[ the whole point of commercials is to get you...,"[the, whole, point, of, commercials, is, to, g...","[[toyota], [tundra]]",[toyota]
3,markz2k,June 2000,"yeah, i agree the point is to get your attent...","[ yeah, i agree the point is to get your atten...","[yeah, ,, i, agree, the, point, is, to, get, y...","[[honda], [tundra, accord]]","[honda, toyota]"
4,bdreggors,June 2000,i've noticed two fallicies in hodna commercia...,[ i've noticed two fallicies in hodna commerci...,"[i, 've, noticed, two, fallicies, in, hodna, c...","[[honda], [accord, accords, civic]]",[honda]
5,floridian,June 2000,the camry commercial that says over and over ...,[ the camry commercial that says over and over...,"[the, camry, commercial, that, says, over, and...","[[], [camry]]",[toyota]
6,fezo,June 2000,absolutely! next chevy will advertise sexy ma...,"[ absolutely!, next chevy will advertise sexy ...","[absolutely, !, next, chevy, will, advertise, ...","[[], [chevy, malibus]]",[chevrolet]
8,microrepair,June 2000,is totally irritated by those toyotathon comm...,[ is totally irritated by those toyotathon com...,"[is, totally, irritated, by, those, toyotathon...","[[toyota], [tundra]]",[toyota]
9,andrew21,June 2000,"...the new ""zoom zoom zoom"" campaign by toyot...","[ ...the new ""zoom zoom zoom"" campaign by toyo...","[..., the, new, ``, zoom, zoom, zoom, '', camp...","[[toyota, bmw], []]","[bmw, toyota]"
10,roadroach,June 2000,i'm starting to get a little irritated by the...,[ i'm starting to get a little irritated by th...,"[i, 'm, starting, to, get, a, little, irritate...","[[], [chevy]]",[chevrolet]


## Lift ratios for associations between brands

## Brands on MDS map

# Task B

What insights can you offer brand managers from your analysis in Task A? Choose two brands that you can offer the most interesting/useful insights for.

# Task C

What are the 5 most frequently mentioned attributes of cars in the discussions? Note that the same attribute may be described by different words – e.g., pick-up and acceleration may both refer to a more general attribute, “performance”. You have to make suitable replacements. Now pick the 5 most frequently mentioned brands. Which attributes are most strongly associated with which of these 5 brands?

# Task D

What advice will you give to a (i) product manager, and (ii) marketing/advertising manager of these brands based on your analysis in Task C?

# Task E

Which is the most aspirational brand in your data in terms of people actually wanting to buy or own? Describe your analysis. What are the business implications for this brand?